In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read and combine data ---------------------------------------------------
final_hcris_v1996 = pd.read_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_v1996.csv')
final_hcris_v2010 = pd.read_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_v2010.csv')

In [3]:
# Create missing variables for columns introduced in v2010
final_hcris_v1996['hvbp_payment'] = np.nan
final_hcris_v1996['hrrp_payment'] = np.nan

In [4]:
# Combine v1996 and v2010 datasets
final_hcris = pd.concat([final_hcris_v1996, final_hcris_v2010])

In [5]:
# Convert date columns to datetime format
for col in ['fy_end', 'fy_start', 'date_processed', 'date_created']:
    final_hcris[col] = pd.to_datetime(final_hcris[col], format='%m/%d/%Y')

In [6]:
# Convert to absolute values
final_hcris['tot_discounts'] = final_hcris['tot_discounts'].abs()
final_hcris['hrrp_payment'] = final_hcris['hrrp_payment'].abs()

In [7]:
# Extract fiscal year and sort
final_hcris['fyear'] = final_hcris['fy_end'].dt.year
final_hcris = final_hcris.sort_values(by=['provider_number', 'fyear']).drop(columns=['year'], errors='ignore')

In [8]:
# Count hospitals per year
hospital_counts = final_hcris.groupby('fyear').size()

In [9]:
# Clean data --------------------------------------------------------------

# Create count of reports by hospital fiscal year
final_hcris['total_reports'] = final_hcris.groupby(['provider_number', 'fyear'])['provider_number'].transform('count')
final_hcris['report_number'] = final_hcris.groupby(['provider_number', 'fyear']).cumcount() + 1

In [12]:

# Identify hospitals with one report per year
unique_hcris1 = final_hcris[final_hcris['total_reports'] == 1].drop(columns=['report', 'total_reports', 'report_number', 'npi', 'status'], errors='ignore')
unique_hcris1['source'] = 'unique reports'
print(unique_hcris1)

      provider_number   fy_start     fy_end date_processed date_created  \
3301            10001 2017-10-01 2018-09-30     2021-01-19   2021-01-18   
3151            10005 2017-10-01 2018-09-30     2021-01-15   2021-01-08   
4490            10006 2018-07-01 2019-06-30     2021-06-03   2021-06-02   
3152            10007 2017-10-01 2018-09-30     2021-01-15   2021-01-08   
3499            10008 2018-01-01 2018-12-31     2021-02-01   2021-01-30   
...               ...        ...        ...            ...          ...   
485            673063 2018-08-01 2019-07-31     2020-01-16   2020-01-03   
279            673064 2018-04-01 2019-03-31     2019-09-19   2019-09-09   
23             673065 2017-10-01 2018-09-30     2019-02-05   2019-01-31   
26             673066 2017-11-10 2018-09-30     2019-02-08   2019-02-05   
6012           673067 2018-05-25 2019-05-31     2022-06-08   2022-06-03   

       beds   tot_charges  tot_discounts  tot_operating_exp   ip_charges  ...  \
3301  327.0  1.839

In [11]:
# Identify hospitals with multiple reports per year
duplicate_hcris = final_hcris[final_hcris['total_reports'] > 1].copy()
print(duplicate_hcris)

      report  provider_number  npi   fy_start     fy_end date_processed  \
202   650635            30137  NaN 2018-07-25 2018-12-31     2019-07-10   
5926  717931            30137  NaN 2018-01-01 2018-07-24     2022-05-26   
4     631292            50043  NaN 2018-01-01 2018-02-28     2018-08-21   
2795  683872            50043  NaN 2018-03-01 2018-12-31     2020-12-31   
3     631094            50305  NaN 2018-01-01 2018-02-28     2018-08-21   
2836  683939            50305  NaN 2018-03-01 2018-12-31     2020-12-31   
2     631016            50523  NaN 2018-01-01 2018-02-28     2018-08-21   
2856  683971            50523  NaN 2018-03-01 2018-12-31     2020-12-31   
441   661471            61326  NaN 2018-01-01 2018-03-31     2019-12-19   
810   667994            61326  NaN 2018-04-01 2018-06-30     2020-04-23   
4478  691659           100211  NaN 2017-10-01 2018-03-31     2021-06-02   
4952  695916           100211  NaN 2018-04-01 2018-09-30     2021-08-18   
44    644255           10

In [13]:
# Calculate elapsed time between fy start and fy end for hospitals with multiple reports
duplicate_hcris['time_diff'] = (duplicate_hcris['fy_end'] - duplicate_hcris['fy_start']).dt.days
duplicate_hcris['total_days'] = duplicate_hcris.groupby(['provider_number', 'fyear'])['time_diff'].transform('sum')

In [14]:
# Hospitals where total days < 370, sum values
unique_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] < 370].groupby(['provider_number', 'fyear']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris2['source'] = 'total for year'

In [15]:
# Hospitals with reports exceeding 370 days
duplicate_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] >= 370].copy()
duplicate_hcris2['max_days'] = duplicate_hcris2.groupby(['provider_number', 'fyear'])['time_diff'].transform('max')
duplicate_hcris2['max_date'] = duplicate_hcris2.groupby(['provider_number', 'fyear'])['fy_end'].transform('max')

In [16]:
# Primary report selection
unique_hcris3 = duplicate_hcris2[(duplicate_hcris2['max_days'] == duplicate_hcris2['time_diff']) &
                                  (duplicate_hcris2['time_diff'] > 360) &
                                  (duplicate_hcris2['max_date'] == duplicate_hcris2['fy_end'])]
unique_hcris3 = unique_hcris3.drop(columns=['report', 'total_reports', 'report_number', 'npi', 'status', 'max_days', 'time_diff', 'total_days', 'max_date'], errors='ignore')
unique_hcris3['source'] = 'primary report'

In [18]:
# Remaining hospitals with reports covering more than one full year
duplicate_hcris3 = duplicate_hcris2[~duplicate_hcris2.index.isin(unique_hcris3.index)].copy()
duplicate_hcris3['time_diff'] = duplicate_hcris3['time_diff'].astype(int)
duplicate_hcris3['total_days'] = duplicate_hcris3['total_days'].astype(int)

cols_to_weight = ['tot_charges', 'tot_discounts', 'tot_operating_exp', 'ip_charges',
                  'icu_charges', 'ancillary_charges', 'tot_discharges', 'mcare_discharges',
                  'mcaid_discharges', 'tot_mcare_payment', 'secondary_mcare_payment',
                  'hvbp_payment', 'hrrp_payment']

duplicate_hcris3[cols_to_weight] = duplicate_hcris3[cols_to_weight].mul(
    duplicate_hcris3['time_diff'] / duplicate_hcris3['total_days'], axis=0)

In [19]:
# Weighted average
unique_hcris4 = duplicate_hcris3.groupby(['provider_number', 'fyear']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris4['source'] = 'weighted_average'

In [20]:
# Combine final datasets
final_hcris_data = pd.concat([unique_hcris1, unique_hcris2, unique_hcris3, unique_hcris4])
final_hcris_data = final_hcris_data.rename(columns={'fyear': 'year'}).sort_values(by=['provider_number', 'year'])

In [21]:
# Save final data ---------------------------------------------------------
final_hcris_data.to_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_Data.csv', index=False)